In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import tensorflow as tf


In [4]:
## Loading the Churn Modelling Dataset
churn_df = pd.read_csv("Churn_Modelling.csv")
churn_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Data Pre-Processing
## Dropping /eliminating columns which are irrelevant to problem Statement
churn_df = churn_df.drop(['RowNumber','CustomerId','Surname'], axis = 1)
churn_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Converting categorical variables to Encoding
gender_encoder = LabelEncoder()
churn_df["Gender"]=gender_encoder.fit_transform(churn_df["Gender"])
churn_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
## Converting categorical column "Geography" using one hot encoding
onehotencoder_geo = OneHotEncoder()
geo_encoder = onehotencoder_geo.fit_transform(churn_df[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
### names of categories in Geography column
onehotencoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
## Converting the geography column array to dataframe
encoded_geo_df = pd.DataFrame(geo_encoder.toarray(),columns=onehotencoder_geo.get_feature_names_out(["Geography"]))
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
## Combining the one hot encoder column to the main dataframe
churn_df = pd.concat([churn_df.drop("Geography", axis = 1),encoded_geo_df], axis = 1)
churn_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
with open("gender_encoder.pkl","wb") as file:
    pickle.dump(gender_encoder, file)

with open("onehotencoder_geo.pkl","wb") as file:
    pickle.dump(onehotencoder_geo,file)


In [12]:
## Dividing the dataset into independent and dependent features

X = churn_df.drop('Exited', axis = 1)
y = churn_df['Exited']

## Splitting the data into training and testing data

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=41)


## Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)





In [13]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

Implementation of Artificial Neural Nework

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [15]:
X_train.shape[1]

12

In [16]:
## Building an Artificial Neural Networks Model
ann_model = Sequential([
    Dense(62,activation= "relu", input_shape = (X_train.shape[1],)), ## First Hiddenlayer connected with input layer
    Dense(31, activation = "relu"), ## second hidden layer
    Dense(1,activation= "sigmoid") ## Output Layer


])

d:\Ann_Classification_project\venvANN\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
ann_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 62)             │           806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 31)             │         1,953 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            32 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,791 (10.90 KB)

 Trainable params: 2,791 (10.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

import tensorflow
optimizer = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000001EE868B7240>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [51]:
## Compiling the ANN Model
ann_model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=['accuracy'])

In [63]:
## Setting up tensorboard
log_dir = "logs"  + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [53]:
## Setting up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15)

In [54]:
## Training the model
training_ann = ann_model.fit(
    X_train,y_train,validation_data=(X_test,y_test), epochs=200,
    callbacks=[tensorflow_callback,early_stopping_callback]
    
)

Epoch 1/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8733 - loss: 0.3036 - val_accuracy: 0.8605 - val_loss: 0.3386
Epoch 2/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8785 - loss: 0.3020 - val_accuracy: 0.8595 - val_loss: 0.3365
Epoch 3/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8734 - loss: 0.2992 - val_accuracy: 0.8610 - val_loss: 0.3392
Epoch 4/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8774 - loss: 0.2974 - val_accuracy: 0.8690 - val_loss: 0.3338
Epoch 5/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8774 - loss: 0.2957 - val_accuracy: 0.8565 - val_loss: 0.3522
Epoch 6/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8750 - loss: 0.2999 - val_accuracy: 0.8600 - val_loss: 0.3409
Epoch 7/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8804 - loss: 0.2934 - val_accuracy: 0.8575 - val_loss: 0.3469
Epoch 8/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8801 - loss: 0.2930 - val_accu

In [55]:
ann_model.save("Ann_model.h5")

In [56]:
## Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [60]:
%reload_ext tensorboard

In [61]:
%tensorboard --log_dir logs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2026-02-27 13:12:58.131354: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-27 13:13:03.131733: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [59]:
import os
os.listdir("logs")

['fit20260226-131938',
 'fit20260227-112824',
 'fit20260227-125819',
 'fit20260227-130944']

In [62]:
os.listdir("logs")

['fit20260226-131938',
 'fit20260227-112824',
 'fit20260227-125819',
 'fit20260227-130944']

In [35]:
%tensorboard --logdir logs --port 6007

In [ ]:
 ##tensorboard --logdir logs --host 127.0.0.1 --port 6012  